<img src="https://mfr.osf.io/export?url=https://osf.io/69jyx/?direct%26mode=render%26action=download%26public_file=True&initialWidth=684&childId=mfrIframe&parentTitle=OSF+%7C+banner.png&parentUrl=https://osf.io/69jyx/&format=1200x1200.jpeg" style="width:600px;"></img>

<center> <h1> ⚠️ </h1> <center>
<center> <h3 style="color:purple;"> Spatial transformations in this repository are for <u >demonstration only</u>!</h3></center>

<center> <h3 style="color:red;">I did not calculate the exact transformation from <code>dwi</code> to <code>MTR</code> space! </h3></center><br>


<center> The fibers are roughly aligned with the <code>MTR</code> volume just for the sake of demonstration. Despite that the interpolation algorithm (<code>idw3dInterp</code>) is sensible, due to imperfect alignment, <code>MTR</code> values are likely to be misregistered on the tracked corpus callosum fibers </center>

## Octave is a bit verbose with warnings, let's silence them for now

In [ ]:
warning ("off","all")


### Let's start by `initEnv` to 
* Add everything we need to the Octave's search path
* Load Octave packages we need

In [ ]:
# we need navigate to the upper directory as this notebook is placed in a folder
cd('..');
# Initialize
initEnv;

### Now we will use the data from the `Eda_Data` directory
* Let's use S2 

In [ ]:
dataDir = [pwd '/Eda_Data/S2/'];

## Calling our first function `getData` 
* `mtOn.nii`
* `mtOff.nii`
* `CC_50.trk` (Less number of tracts for Octave)

In [ ]:
[data,header]  = getData([dataDir 'mtOn.nii'],[dataDir 'mtOff.nii'],[dataDir 'CC_50.trk']);

In [ ]:
%plot inline -w 800 -h 300
subplot(1,2,1); imagesc(data.mtOn(:,:,30)); title('MTon image');
subplot(1,2,2); imagesc(data.mtOff(:,:,30)); title('MToff image');

### Read affine transformation with `readTransformation` function

In [ ]:
affineTransform = readTransformation([dataDir 'transform.txt']);

### Use `transformFibers` to nicely align fibers with structural images

In [ ]:
transformedFibers = transformFibers(data.fibers,header.mtOn.origin,affineTransform);

### Now `plotSlice` and  `trk_plot` to see if fibers are aligned with the image
* This one may take a bit longer 
* Octave 3D surf plot does not look good in Jupyter Notebooks. But don't worry, we have a solution!

In [ ]:
plotSlice(data.mtOn(:,:,30),header.mtOn,30,'hold','on')
trk_plot(header.fibers,transformedFibers);

## Another good software practice is not to reinvent the wheel

* Remember that this Docker image was based on `qmrlab/octjc:v2.4.0`, which comes with `qMRLab`, a tool to do quantiative MRI analysis! 

* To calculate `MTR`, let's use `qMRLab`. You can see the [`MTR` documentation](https://qmrlab.readthedocs.io/en/master/mt_ratio_batch.html).

<center> <h1> 🐳 </h1> <center>
    
<h3 style="color:red;"> If you are executing this notebook in <code>Binder</code> or using <code>Docker</code>, calculate the <code>MTR</code> in the cell below using <code>qMRLab</code>. <br><br> Otherwise, proceed to the next red-colored text :) </h3>


#### Calculate `MTR` image using `qMRLab`

In [ ]:
model = mt_ratio;
% Load data into qMRLab
Data.MToff = data.mtOn;
Data.MTon = data.mtOff;
FitResults = FitData(Data,model,0);

#### Display `MTR` image

In [ ]:
qMRshowOutput(FitResults,data,model);

#### Save `MTR` image in the coordinates of  `mtOn.nii`  to the directory  `dataDir`

In [ ]:
FitResultsSave_nii(FitResults, [dataDir 'mtOn.nii'], dataDir);

In [ ]:
MTR = FitResults.MTR;

Remember that `MTR` is just one of the many qMRI methods! To see what else is possible with qMRLab, you can visit [here](https://qmrlab.org)!

<center> <h1> 💻 </h1> <center>
    
<h3 style="color:red;"> If you are NOT executing this notebook in <code>Binder</code> or using <code>Docker</code>, calculate the <code>MTR</code> in the cell below </h3>



In [ ]:
MTR = getMtr(data.mtOn,data.mtOff);

## Perform inverse distance weighted interpolation of  `MTR`  along corpus callosum tracts


In [ ]:
[interpolatedHeader,interpolatedFibers] = idw3dInterp(header.fibers,transformedFibers,header.mtOn,MTR);

## Save `MTR` weighted fibers as `*.trk` file
* You can download them to your local and visualize using your favorite viewer.
* **You will find the fibers in the same directory you loaded the input data from.**

In [ ]:
trk_write(interpolatedHeader,interpolatedFibers,[dataDir 'cc_mtr.trk']);

## Save ` MTR` weighted fibers in MATLAB compatible `*.mat` format
* We will read them in another notebook

**Note:** To be able to read `mat` files saved by Octave in MATLAB, you need to pass **`-mat7-binary`** argument

In [ ]:
save 'mtrFibers.mat' 'interpolatedHeader' 'interpolatedFibers'

## Display `MTR` fibers 


> Inline plot with scalars is going to fail in Octave due to a bug in `trk_plot`, but no worries, I prepared you another notebook for interactive visualization! 

#### See the [interactive visualization notebook](interactive_vis.ipynb)